In [82]:
import panel as pn
pn.extension('plotly')
from panel.interact import interact, interactive, fixed, interact_manual
from panel import widgets
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv
import numpy as np
from datetime import date
import datetime
from datetime import timedelta
import requests
import alpaca_trade_api as tradeapi 


%matplotlib inline

In [83]:
# Load env vars
load_dotenv()

True

In [84]:
# Get API keys
coin_api_key = os.getenv('COIN_API_KEY') 
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
access_token = os.getenv("CQ_API_KEY")

In [85]:
# Load Headers
headers = { 'Authorization': 'Bearer ' + access_token }

In [86]:
# Common date params
start_date = '2020-10-01'
end_date = '2021-01-01'

In [87]:
# Dashboard constants
PANEL_WIDTH = 500
PANEL_HEIGHT = 400

In [88]:
def get_stock_OHLCV(api_key, secret_key, ticker, date_start, date_end, period) :
    '''
    Historical OHLCV for stocks using Alpaca API.

            Parameters:
                    api_key (string): Alpaca api key
                    secret_key (string): Alpaca secret key
                    ticker (string): Ticker - ex) 'SPY'
                    date_start (string): example) '2015-01-01'
                    date_end (string): example) '2015-01-01'
                    period (string): example) '1D'

            Returns:
                    DateFrame with columns: ['date','open','high','low','close','volume']
    '''

    # Init Alpaca SDK
    api = tradeapi.REST(
        api_key,
        secret_key,
        api_version="v2")

    # Init start/end dates
    start_date = pd.Timestamp(date_start, tz="America/New_York").isoformat()
    end_date = pd.Timestamp(date_end, tz="America/New_York").isoformat()

    # Make API call
    raw_df = api.get_barset(
        [ticker],
        period,
        start=start_date,
        end=end_date).df

    # Init new DataFrame
    df = pd.DataFrame()
    df['open'] = raw_df[ticker]['open']
    df['high'] = raw_df[ticker]['high']
    df['low'] = raw_df[ticker]['low']
    df['close'] = raw_df[ticker]['close']
    df['volume'] = raw_df[ticker]['volume']

    # Parse date
    df['date'] = raw_df.index.date

    # Reset index
    df.reset_index(inplace=True)

    # Reorder columns
    df = df[['date','open','high','low','close','volume']]

    return df

In [89]:
def get_crypto_OHLCV(api_key, ticker, date_start, date_end, period) :
    '''
    Historical OHLCV for crypto using OpenAPI.io.

            Parameters:
                    api_key (string): OpenAPI.io api key
                    ticker (string): Ticker - ex) 'BTC'
                    date_start (string): example) '2015-01-01'
                    date_end (string): example) '2015-01-01'
                    period (string): example) '1DAY'

            Returns:
                    DateFrame with columns: ['date','open','high','low','close','volume']
    '''
    # CoinAPI.io REST url for historical OHLCV
    url = f'https://rest.coinapi.io/v1/ohlcv/{ticker}/USD/history?apikey={api_key}&period_id={period}&time_start={date_start}&time_end={date_end}&limit=100000'
    # url = f'https://rest.coinapi.io/v1/ohlcv/{ticker}/USD/history?apikey={api_key}&period_id={period}&time_start={date_start}&time_end={date_end}&limit=100'
    
    # List of dictionary objects
    results = requests.get(url).json()

    # Init dataframe from raw results
    df = pd.DataFrame(
        results, 
        columns=['time_period_end','price_open','price_high','price_low','price_close','volume_traded'])

    # Parse date from string
    df['time_period_end'] = pd.to_datetime(df['time_period_end']).dt.date

    # Rename columns
    df.rename(
        columns = {'time_period_end':'date',
            'price_open':'open',
            'price_high':'high',
            'price_low':'low',
            'price_close':'close',
            'volume_traded':'volume'}, 
        inplace = True)

    # Save to file
    #df.to_csv(f'../data/OHLCV/{ticker}_ohlcv_{date_start}-{date_end}_v2.csv', header=None, index=None, sep=',', mode='a')

    return df

In [90]:
def get_pct_chg_for_OHLCV(ohlcv_data) :
    '''
    Helper method to convert OHLCV data to pct_chg.

            Parameters:
                    ohlcv_data (DataFrame): OHLCV data (from methods above)

            Returns:
                    DateFrame with column: ['close']
    '''
    returns_df = ohlcv_data[['close']]    
    returns_df.index = ohlcv_data['date']
    returns_df = returns_df.pct_change()
    return returns_df

In [91]:
def get_daily_returns(start_date, end_date, coin_api_key, alpaca_api_key, alpaca_secret_key) :
    '''
    Get daily returns for pre-selected assets

            Parameters:
                    start_date (string): start date
                    end_date (string): start date

            Returns:
                    Panel.pane object for use in dashboard
    '''
    # Get closing data
    btc_df = get_crypto_OHLCV(coin_api_key, 'BTC', start_date, end_date, '1DAY')
    eth_df = get_crypto_OHLCV(coin_api_key, 'ETH', start_date, end_date, '1DAY')
    xrp_df = get_crypto_OHLCV(coin_api_key, 'XRP', start_date, end_date, '1DAY')
    link_df = get_crypto_OHLCV(coin_api_key, 'LINK', start_date, end_date, '1DAY')
    ltc_df = get_crypto_OHLCV(coin_api_key, 'LTC', start_date, end_date, '1DAY')
    xlm_df = get_crypto_OHLCV(coin_api_key, 'XLM', start_date, end_date, '1DAY')
    spy_df = get_stock_OHLCV(alpaca_api_key, alpaca_secret_key, 'SPY', start_date, end_date, '1D')
    gld_df = get_stock_OHLCV(alpaca_api_key, alpaca_secret_key, 'GLD', start_date, end_date, '1D')

    # Calculate daily returns
    btc_returns_df = get_pct_chg_for_OHLCV(btc_df)
    eth_returns_df = get_pct_chg_for_OHLCV(eth_df)
    xrp_returns_df = get_pct_chg_for_OHLCV(xrp_df)
    link_returns_df = get_pct_chg_for_OHLCV(link_df)
    ltc_returns_df = get_pct_chg_for_OHLCV(ltc_df)
    xlm_returns_df = get_pct_chg_for_OHLCV(xlm_df)
    spy_returns_df = get_pct_chg_for_OHLCV(spy_df)
    gld_returns_df = get_pct_chg_for_OHLCV(gld_df)

    # Combine daily returns
    combined_daily_returns = pd.concat(
        [btc_returns_df, eth_returns_df, xrp_returns_df, ltc_returns_df, spy_returns_df, gld_returns_df], 
        axis="columns", 
        join="inner")

    combined_daily_returns = combined_daily_returns.dropna()

    # Rename columns
    combined_daily_returns.columns = ['BTC','ETH','XRP','LTC','SPY','GLD']

    # Ensure we are sorted by date (index)
    combined_daily_returns.sort_index()

    return combined_daily_returns

In [92]:
def get_cumulative_returns_plot(daily_returns) :
    
    # Calculate cumulative returns     
    combined_cumulative_returns = (1 + daily_returns).cumprod()

    # Plot with Plotly Express     
    cum_returns_plot = px.line(
        combined_cumulative_returns,
        title ='Cumulative Returns',
        labels={'date':'Date'},
        width=PANEL_WIDTH*2,
        height=PANEL_HEIGHT)
  
    return pn.panel(cum_returns_plot)

In [93]:
def get_std_dev_plot(daily_returns) :
    
    rolling_stddev_all = daily_returns.rolling(window=21).std().dropna()

    # Plot with Plotly Express     
    rolling_stddev_plot = px.line(
        rolling_stddev_all,
        title ='Rolling Standard Deviation',
        labels={'date':'Date'},
        width=PANEL_WIDTH*2,
        height=PANEL_HEIGHT)
  
    return pn.panel(rolling_stddev_plot)

In [94]:
def get_netflow_data(ticker,fromdate,limit):
    '''
    Returns on-chain data (Netflow)n in new df
        Parameters:
            ticker (string) = Ticker for Desired Coin
            fromdate (string) = Starting date of period desired
            limit (int) = number of data points     
    '''
    #Initialize dotenv function, pull API key, build URL
    headers = {'Authorization': 'Bearer ' + access_token}
    URL = f'https://api.cryptoquant.com/v1/{ticker}/exchange-flows/netflow?exchange=all_exchange&window=day&from={fromdate}&limit={limit}'

    # Initialize Dictionary
    response = requests.get(URL, headers=headers)
    
    netflow_data = response.json()

    #Transform to Dataframe and adjust data type
    netflow_df = pd.DataFrame(data=netflow_data['result']['data'])
    netflow_df['date'] = pd.to_datetime(netflow_df['date'])
    netflow_df['netflow_total'] = netflow_df['netflow_total'].astype(str).astype('float64')
   
    return netflow_df

In [95]:
netflow_btc_df = get_netflow_data("btc","20190101","100")
netflow_btc_df.head(5)

,date,netflow_total
0,2021-01-30,5216.551173
1,2021-01-29,17413.018375
2,2021-01-28,-6092.090538
3,2021-01-27,-10347.658278
4,2021-01-26,-2644.317802


In [96]:
def get_price_data(ticker,fromdate,limit):
    '''
    Returns market data (ticker) in new df
        Parameters:
            ticker (string) = Ticker for Desired Coin
            fromdate (string) = Starting date of period desired
            limit (int) = number of data points     
    '''
    #Get Data & Initialize Dictionary
    URL2 = f'https://api.cryptoquant.com/v1/{ticker}/market-data/price-usd?window=day&from={fromdate}&limit={limit}'
    response2 = requests.get(URL2, headers=headers)
    price_data = response2.json()
    
    #Transform to Dataframe and adjust data type
    price_df = pd.DataFrame(data=price_data['result']['data'])
    price_df = price_df.drop(['price_usd_high','price_usd_low','price_usd_open'], axis = 1)
    price_df['date'] = pd.to_datetime(price_df['date'])
    price_df['price_usd_close'] = price_df['price_usd_close'].astype(str).astype('float64')
    price_df = price_df.dropna()

    return price_df

In [97]:
price_btc_df = get_price_data("btc","20190101","100")
price_btc_df.head(5)

,date,price_usd_close
0,2021-01-30,34286.360000
1,2021-01-29,34244.506667
2,2021-01-28,33383.746667
3,2021-01-27,30385.111667
4,2021-01-26,32489.385000


In [98]:
def get_mvrv_data(ticker,fromdate,limit):
    '''
    Returns marketcap data in new df
        Parameters:
            ticker (string) = Ticker for Desired Coin
            fromdate (string) = Starting date of period desired
            limit (int) = number of data points     
    '''
    #Get Data & Initialize Dictionary
    URL3 = f'https://api.cryptoquant.com/v1/{ticker}/market-data/capitalization?window=day&from={fromdate}&limit={limit}'
    response3 = requests.get(URL3, headers=headers)
    mvrv_data = response3.json()

    #Transform to Dataframe and adjust data type
    mvrv_data = pd.DataFrame(data=mvrv_data['result']['data'])
    mvrv_data['mvrv_score'] = (mvrv_data['market_cap']/mvrv_data['realized_cap'])
    mvrv_data['date'] = pd.to_datetime(mvrv_data['date'])
    mvrv_data = mvrv_data.sort_values(by=['date'],ascending=False)
    mvrv_data = mvrv_data.drop(['average_cap', 'delta_cap', 'market_cap','realized_cap','thermo_cap'], axis = 1)

    return mvrv_data

In [99]:
mvrv_btc_df = get_mvrv_data("btc","20190101","100")
mvrv_btc_df.head(5)

,date,mvrv_score
0,2021-01-30,2.632662
1,2021-01-29,2.635438
2,2021-01-28,2.591286
3,2021-01-27,2.367614
4,2021-01-26,2.544537


In [100]:
def get_pearsons_coefficient(series1,series2):
    ''' 
    Returns pearsons coefficient (int) between 2 dataframes columns.
    Parameters: 
        series1 (object): First series to compare
        series2 (object): Second series to compare
    '''
    #formula to calculate covariance
    covariance = cov(series1,series2)
    corr, _ = pearsonr(series1,series2)
    
    return corr

In [101]:
def netflow_price_plot():
    fig3 = make_subplots(specs=[[{'secondary_y':True}]])

    fig3.add_trace(
        go.Scatter(mode = 'lines',x=netflow_btc_df['date'],y=netflow_btc_df['netflow_total'], name = 'All Exchange Netflow'),
        secondary_y=False 
    )

    fig3.add_trace(
        go.Scatter(mode = 'lines',x=price_btc_df['date'], y =price_btc_df['price_usd_close'], name = 'BTC Price'),
        secondary_y=True 
    )

    fig3.update_layout(
        height = 600,width = 1500,
        title_text = 'BTC Price & Netflow Data'
    )

    fig3.update_xaxes(title_text = 'Date')

    fig3.update_yaxes(title_text='Exchange Netflow', secondary_y= False)
    fig3.update_yaxes(title_text='BTC Market Price', secondary_y= True)



    #return fig3.show()
    return pn.panel(fig3)

In [102]:
def mvrv_price_plot():
    fig3 = make_subplots(specs=[[{'secondary_y':True}]])

#fig3=go.Figure()

    fig3.add_trace(
        go.Scatter(mode = 'lines',x=mvrv_btc_df['date'],y=mvrv_btc_df['mvrv_score'], name = 'MVRV Ratio'),
        secondary_y=False 
    )

    fig3.add_trace(
        go.Scatter(mode = 'lines',x=price_btc_df['date'], y =price_btc_df['price_usd_close'], name = 'BTC Price'),
        secondary_y=True 
    )

    fig3.update_layout(
        height = 600,width = 1500,
        title_text = 'BTC Price & MVRV Ratio'
    )

    fig3.update_xaxes(title_text = 'Date')

    fig3.update_yaxes(title_text='MVRV Ratio', secondary_y= False)
    fig3.update_yaxes(title_text='BTC Market Price', secondary_y= True)



    #return fig3
    return pn.panel(fig3)

In [103]:
# Get Daily returns (shared for multiple plots)
daily_returns = get_daily_returns(start_date, end_date, coin_api_key, alpaca_api_key, alpaca_secret_key) 

In [104]:
# Define tab layout
dashboard = pn.Tabs(
    ('Cumulative Returns', pn.Column(get_cumulative_returns_plot(daily_returns))),
    ('Standard Deviation', pn.Column(get_std_dev_plot(daily_returns))),
    ('MVRV Ratio',pn.Column(mvrv_price_plot())),
    ('Netflow Ratio',pn.Column(netflow_price_plot())))

In [107]:
print(f'Current date range: {start_date} - {end_date}')

Current date range: 2020-10-01 - 2021-01-01


In [105]:
dashboard.servable()

Tabs
    [0] Column
        [0] Plotly(Figure)
    [1] Column
        [0] Plotly(Figure)
    [2] Column
        [0] Plotly(Figure)
    [3] Column
        [0] Plotly(Figure)